In [ ]:
from jinja2 import Template
import markdown
from weasyprint import HTML, CSS

# 1. Markdown input (content only)
md_content = """
# Report Title
<!-- SUBTITLE: 20250718 -->
<!-- PAGEBREAK -->

## Section 1

Some more content here.
"""

# 2. Convert Markdown → HTML fragment

md_content = md_content.replace("<!-- PAGEBREAK -->", "<div style='page-break-before: always;'></div>")
md_content = md_content.replace("<!-- SUBTITLE:", "<p class='subtitle'>").replace("-->", "</p>")
html_body = markdown.markdown(md_content)

# 3. Apply HTML template without embedded style
template = Template("""
<html>
<head></head>
<body>{{ body }}</body>
</html>
""")
full_html = template.render(body=html_body)

# 4. Define clean external CSS
css = CSS(string="""
@page {
    /* size: A4; */
    size: 11in 8.5in;
    margin: 1in;
}
body {
    font-family: font-family: 'Aptos', 'Arial', sans-serif;
    line-height: 1.6;
    font-size: 12pt;
}
h1 {
    text-align: center;
    color: black;
    margin-top: 3in;
    font-family: 'Palatino Linotype', Palatino, serif;
}
.subtitle {
    text-align: center;
    color: black;
    font-family: 'Palatino Linotype', Palatino, serif;
}
h2, h3 {
    color: black;
    font-family: 'Palatino Linotype', Palatino, serif;
}
strong {
    font-weight: bold;
}
em {
    font-style: italic;
}
""")

# 5. Generate PDF with clean content/style separation
HTML(string=full_html).write_pdf("output.pdf", stylesheets=[css])